In [1]:
import pandas
import numpy as np
import sklearn as sk

from sklearn.preprocessing import OneHotEncoder
import time
import pickle

%matplotlib inline

In [5]:
dataset_name = "data/raw/data_1.csv"
data = pandas.read_csv(dataset_name).drop(["id1", "id2"], axis=1)
data.head()

,id3,user_id,date
0,714,464300,1
1,590,1029729,1
2,703,861305,1
3,803,733016,1
4,219,448027,1


In [43]:
data = data[data["user_id"].isin(data["user_id"].unique()[:5379])]

In [11]:
ohe = OneHotEncoder()

ohe.fit(data[["id3"]])
id3_columns = ["id2_%s" % i for i in range(data["id3"].unique().shape[0])] 

x_column_names = ["user_id"] + id3_columns

In [17]:
def vectorize_user(log):
    # Преобразуем все id в one hot
    values = ohe.transform(log[["id3"]])
    
    #суммируем все вектора и получаем вектор истории просмотров
    values = np.array(values.sum(axis=0))[0].astype("float")
    values = values/values.sum()
    
    user_id = np.array([log["user_id"].get_values()[0]])
    vector = np.concatenate([user_id, values])
    
    user_vector = pandas.DataFrame(columns=x_column_names, data=[vector])
        
    return user_vector
    

In [33]:
def get_user_item_matrix(log):
    start_time = time.time()
    users = log["user_id"].unique()
    user_item_mtrx = pandas.DataFrame(columns=x_column_names) 
    
    for user in users:
        row = vectorize_user(log[log["user_id"] == user])
        user_item_mtrx = pandas.concat((user_item_mtrx, row))
    print("Result time:", time.time()-start_time)
    return user_item_mtrx

In [44]:
usr_item_mtrx = get_user_item_matrix(data)

Result time: 164.75401544570923


In [47]:
usr_item_mtrx.to_csv("usr_item_mtrx.csv", index=False)

In [67]:
from annoy import AnnoyIndex
ann = AnnoyIndex(usr_item_mtrx.shape[1]-1)

In [68]:
mtrx = usr_item_mtrx.get_values()
for i, vec in enumerate(mtrx):
    ann.add_item(i, vec[1:])

In [69]:
ann.build(10)

True

In [165]:
ans = []
k = [i for i in range(mtrx.shape[1])]

for i, vec in enumerate(mtrx):
    a = mtrx[(ann.get_nns_by_item(i, 5)[1:])].mean(axis=0)[vec==0]
    a = sorted(list(zip(a.tolist(), k)), reverse=True)[:5]
    a = [int(i[1]) for i in a]
    ans.append(a)
ans = np.array(ans)

In [166]:
users = usr_item_mtrx["user_id"]
users = users.get_values().reshape((users.shape[0],1))

In [167]:
ans = np.concatenate([users, ans], axis=1).astype("int")

In [169]:
submit = pandas.DataFrame(data=ans, columns=["user_id", "id3_1", "id3_2", "id3_3", "id3_4", "id3_5"])

In [171]:
submit.head()

,user_id,id3_1,id3_2,id3_3,id3_4,id3_5
0,464300,639,394,418,260,628
1,1029729,94,149,426,413,393
2,861305,68,59,489,461,645
3,733016,343,297,98,87,81
4,448027,45,315,180,459,187


In [174]:
submit.to_csv("submit.csv", index=False)

In [178]:
sample = pandas.read_csv("track1_sample_submission.csv")

In [181]:
sample[sample["user_id"].isin(users[:,0])].shape

(222, 6)

In [182]:
users.shape

(5379, 1)

In [185]:
submit.shape

(5379, 6)

In [2]:
data = pandas.read_csv("train.csv")

In [3]:
submit = pandas.read_csv("submit.csv")

In [4]:
usrs = data["user_id"].unique()

In [13]:
usrs = usrs[np.invert(np.isin(usrs, submit["user_id"]))]

In [16]:
data = pandas.DataFrame(columns=["user_id", "id3_1", "id3_2", "id3_3", "id3_4", "id3_5"])
data["user_id"] = usrs

In [23]:
data["id3_1"] = submit.median()[1]
data["id3_2"] = submit.median()[2]
data["id3_3"] = submit.median()[3]
data["id3_4"] = submit.median()[4]
data["id3_5"] = submit.median()[5]

In [25]:
submit = pandas.concat([submit, data])

In [31]:
submit = submit.astype("int")

In [40]:
submit[:53979].to_csv("submit2.csv", index=False)

In [41]:
submit[:53979]

,user_id,id3_1,id3_2,id3_3,id3_4,id3_5
0,464300,639,394,418,260,628
1,1029729,94,149,426,413,393
2,861305,68,59,489,461,645
3,733016,343,297,98,87,81
4,448027,45,315,180,459,187
5,913436,135,332,863,412,518
6,5372,359,537,290,512,455
7,325474,69,526,592,63,454
8,489465,383,402,250,175,406
9,124475,550,232,527,222,594
